In [25]:
!pip install datasets pandas torch openpyxl banglanltk bangla-stemmer gensim pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.1 MB/s eta 0:00:00


In [26]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from datasets import Dataset
import numpy as np
import wandb
from bangla_stemmer.stemmer import stemmer
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models

In [27]:
# Load the dataset
df = pd.read_excel("dataset_rv.xlsx")
stopwords_df = pd.read_excel('stopwords_bangla.xlsx')

stopwords_df = stopwords_df[['words']]

# Map string labels to numerical labels
label_map = {"outrage": 0, "despair": 1, "hope": 2}
df["Sentiment"] = df["Sentiment"].map(label_map)

# Rename the column to text
df.rename(columns={'Headline':'text', 'Sentiment':'labels'}, inplace=True)
stopwords_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,words
0,অই
1,অগত্যা
2,অত: পর
3,অতএব
4,অথচ


In [28]:
def english_to_bangla_number_text(number):
    """Converts an English number to Bangla textual representation."""
    bangla_numbers = {
        0: "শূন্য", 1: "এক", 2: "দুই", 3: "তিন", 4: "চার",
        5: "পাঁচ", 6: "ছয়", 7: "সাত", 8: "আট", 9: "নয়",
        10: "দশ", 11: "এগারো", 12: "বারো", 13: "তেরো", 14: "চৌদ্দ",
        15: "পনেরো", 16: "ষোলো", 17: "সতেরো", 18: "আঠারো", 19: "উনিশ",
        20: "বিশ", 21: "একুশ", 22: "বাইশ", 23: "তেইশ", 24: "চব্বিশ",
        25: "পঁচিশ", 26: "ছাব্বিশ", 27: "সাতাশ", 28: "আটাশ", 29: "ঊনত্রিশ",
        30: "ত্রিশ", 31: "একত্রিশ", 32: "বত্রিশ", 33: "তেত্রিশ", 34: "চৌত্রিশ",
        35: "পঁত্রিশ", 36: "ছত্রিশ", 37: "সাঁইত্রিশ", 38: "আটত্রিশ", 39: "ঊনচল্লিশ",
        40: "চল্লিশ", 41: "একচল্লিশ", 42: "বিয়াল্লিশ", 43: "তেতাল্লিশ", 44: "চুয়াল্লিশ",
        45: "পঁইয়াল্লিশ", 46: "ছিয়াল্লিশ", 47: "সাতচল্লিশ", 48: "আটচল্লিশ", 49: "ঊনপঞ্চাশ",
        50: "পঞ্চাশ", 51: "একান্ন", 52: "বাহান্ন", 53: "তিপ্পান্ন", 54: "চুয়ান্ন",
        55: "পঞ্চান্ন", 56: "ছাপ্পান্ন", 57: "সাতান্ন", 58: "আটান্ন", 59: "ঊনষাট",
        60: "ষাট", 61: "একষট্টি", 62: "বাষট্টি", 63: "তেষট্টি", 64: "চৌষট্টি",
        65: "পঁষট্টি", 66: "ছেষট্টি", 67: "সাতষট্টি", 68: "আটষট্টি", 69: "ঊনসত্তর",
        70: "সত্তর", 71: "একাত্তর", 72: "বাহাত্তর", 73: "তিয়াত্তর", 74: "চুয়াত্তর",
        75: "পঁচাত্তর", 76: "ছিয়াত্তর", 77: "সাতাত্তর", 78: "আটাত্তর", 79: "ঊনআশি",
        80: "আশি", 81: "একাশি", 82: "বিরাশি", 83: "তিরাশি", 84: "চুরাশি",
        85: "পঁচাশি", 86: "ছিয়াশি", 87: "সাতাশি", 88: "আটাশি", 89: "ঊননব্বই",
        90: "নব্বই", 91: "একানব্বই", 92: "বিরানব্বই", 93: "তিরানব্বই", 94: "চুরানব্বই",
        95: "পঁচানব্বই", 96: "ছিয়ানব্বই", 97: "সাতানব্বই", 98: "আটানব্বই", 99: "নিরানব্বই",
    }

    def convert_two_digit_number(n):
        """Handles numbers from 0 to 99 based on direct mapping."""
        if n in bangla_numbers:
            return bangla_numbers[n]
        tens = (n // 10) * 10
        units = n % 10
        return f"{bangla_numbers[tens]} {bangla_numbers[units]}" if units > 0 else bangla_numbers[tens]

    if number < 100:
        return convert_two_digit_number(number)
    elif number < 1000:
        hundreds = number // 100
        remainder = number % 100
        if remainder == 0:
            return f"{bangla_numbers[hundreds]} শত"
        return f"{bangla_numbers[hundreds]} শত {convert_two_digit_number(remainder)}"
    elif number < 100000: #Handle numbers from 1000 to 99,999
        parts = []
        if number >= 1000:
            parts.append(f"{english_to_bangla_number_text(number // 1000)} হাজার")
            number %= 1000
        if number > 0:
            if number >= 100:
              parts.append(english_to_bangla_number_text(number))
            else:
              parts.append(convert_two_digit_number(number))  #handles cases less than 100
        return " ".join(parts)
    elif number < 10000000: #handles numbers from 100,000 to 9,999,999
      parts = []
      if number >= 100000:
          parts.append(f"{english_to_bangla_number_text(number // 100000)} লক্ষ")
          number %= 100000
      if number > 0:
          if number >= 1000:
            parts.append(english_to_bangla_number_text(number))
          elif number > 0:
            if number >= 100:
                parts.append(english_to_bangla_number_text(number))
            else:
              parts.append(convert_two_digit_number(number)) #handle numbers less than 100
      return " ".join(parts)
    else: # Handle numbers >= 10,000,000 (Crore)
        parts = []
        if number >= 10000000:
            parts.append(f"{english_to_bangla_number_text(number // 10000000)} কোটি")
            number %= 10000000
        if number > 0:
            parts.append(english_to_bangla_number_text(number))
        return " ".join(parts)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
import re

def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\u2600-\u26FF"          # miscellaneous symbols
                               u"\u2700-\u27BF"          # dingbats
                               u"\u2000-\u206F"          # general punctuations
                               "]+", flags=re.UNICODE)
    english_pattern = re.compile('[a-zA-Z]+', flags=re.I)

    text = emoji_pattern.sub(r'', text)
    text = english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct += char

    return no_punct

def convert_numbers_to_bangla(text):
    words = text.split()
    converted_words = []
    for word in words:
        if word.isdigit():  # Check if the word is an integer
            bangla_number = english_to_bangla_number_text(int(word))
            converted_words.append(bangla_number)
        else:
            converted_words.append(word)
    return ' '.join(converted_words)

def preprocessing(text):
    text = replace_strings(text)
    text = convert_numbers_to_bangla(text)  # Convert numbers to Bangla
    text = remove_punctuations(text)
    return text

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# Remove Stopwords
stopwords = set(stopwords_df['words'].tolist())
def stopwordRemoval(text):
    return ' '.join([word for word in text.split() if word not in stopwords])

# stemmer function
def stem_text(x):
  stmr = stemmer.BanglaStemmer()
  words=x.split(' ')
  stm = stmr.stem(words)
  words=(' ').join(stm)
  return words

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
df['text'] =df['text'].apply(lambda x: preprocessing(str(x)))
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,labels
0,ঢাকা বিশ্ববিদ্যালয় কোটাবিরোধী আন্দোলনের নেতাকে...,0
1,কোটাবিরোধী আন্দোলনে ঢাকা বিশ্ববিদ্যালয়ের বিএনপ...,2
2,কোটাবিরোধী আন্দোলন আজও জিরো পয়েন্ট অবরোধ করে ...,0
3,সর্বজনীন পেনশন সরকার অনড় আন্দোলন চালিয়ে যাবেন ...,0
4,আজ শনিবার সকাল সাড়ে টার দিকে মহাসড়কের শহর বাইপ...,0


In [32]:
df['text'] =df['text'].apply(lambda x: stopwordRemoval(str(x))) # remove stopwords
df['text'] =df['text'].apply(lambda x: stem_text(str(x))) # stem the text
texts = df['text'].tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied third rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied third rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..


In [33]:
# Tokenize
def tokenized_data(sent):
    tokenized_text = sent.split()
    return tokenized_text

texts = df['text'].tolist()
df['text'] = [tokenized_data(sent) for sent in texts]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
# TF-IDF Feature Extraction with custom tokenizer
texts = [' '.join(tokens) for tokens in df['text']]

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=10000, use_idf=True, tokenizer=lambda x: x.split())
tfidf = tfidf_vectorizer.fit_transform(texts)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [35]:
# Gensim Topic Modeling
# Tokenize the text (already done)
tokenized_texts = df['text'].tolist()

# Create a dictionary
dictionary = Dictionary(tokenized_texts)
dictionary.filter_extremes(no_below=3, no_above=0.90)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]


# Define the number of topics
num_topics = 10

# Train the LDA model
lda_model = LdaModel(corpus,
                   num_topics=num_topics,
                   id2word=dictionary,
                   random_state=42,
                   passes=20,
                   alpha='auto',
                   eta='auto',
                   per_word_topics=True
                    )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
# Function to show topics
def show_topics(lda_model, num_topics=10, top_words=10):
    for topic_id in range(num_topics):
        topic_words = lda_model.show_topic(topic_id, topn=top_words)
        topic_words = " ".join([word for word, prob in topic_words])
        print(f'Topic {topic_id + 1}: {topic_words}')

# Show top words for each topic
print("Top words per topic (using Gensim):")
show_topics(lda_model, num_topics=num_topics)

Top words per topic (using Gensim):
Topic 1: ঢাকা চাঁদা না নেতাকর্মী বিএনপির পায় সেবা সিটি দায় আগ
Topic 2: শেখ গুম সরকার হাসিনা অন্তর্বর্তী ঘটনা সিদ্ধান্ত বিএনপি কমিশন অভিযোগ
Topic 3: আন্দোলনে চোখে পুলিশ শিক্ষার্থী থাকা দাবি নজরুল বারবার গোপন কোটা
Topic 4: হত্যা সরকারি মা চট্টগ্রামে সংগ্রহ ত্রাণ জব্দ টিএসসি অভিযান শিক
Topic 5: পথে সরক সময় আহত ফেনী ত্রাণ চিকিৎসা ফেরার আমলে অভিজ্ঞতা
Topic 6: না পুলিশ চল থানা কার্যক্রম প্রকাশ মেডিকেল কর্মকর্তা কলেজ মহানগর
Topic 7: আন্দোলন পুলিশ ছাত্রজনত হাত মুখে নিজ হাসপাতাল নারী কীভাবে বাড়ির
Topic 8: কোটা সংস্ক আন্দোলন ভারত দল নির্বাচন তৈরি শুক্রব সকাল সরক
Topic 9: আওয়ামী দেশ লীগ ওপর রহমান হক সভাপতি আন্দোলন অধিক অভিযোগ
Topic 10: কালোটাকা পানি উপদেষ্ গুলিবিদ্ধ ফির গতকাল যেভাবে প্রধান দশ বন্যার্ত


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
# Function to calculate coherence score
def calculate_coherence(lda_model, texts, dictionary):
    coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    return coherence_score

# Calculate coherence score
coherence_score = calculate_coherence(lda_model, tokenized_texts, dictionary)
print(f"Coherence score: {coherence_score:.4f}")

# Topic Visualization with pyLDAvis
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence score: 0.4551


In [38]:
# Scikit-learn Topic Modeling

# Define number of topics
num_topics = 10

# Train the LDA Model
lda_sklearn = LatentDirichletAllocation(n_components=num_topics,
                                        random_state=42,
                                        learning_decay=0.7,
                                        max_iter=100,
                                        learning_method='batch')
lda_sklearn.fit(tfidf)

# Function to display top words for each topic
def display_topics(model, feature_names, num_top_words=10):
  for topic_idx, topic in enumerate(model.components_):
    print(f"Topic {topic_idx + 1}:", " ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

# Get feature names
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# Show top words for each topic
print("\nTop words per topic (using Scikit-learn):")
display_topics(lda_sklearn, tfidf_feature_names, num_top_words=10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Top words per topic (using Scikit-learn):
Topic 1: ব্যাংক শিক্ষার্থী পুলিশ বিরুদ্ধে না শেখ আলম পরিব ইসলামী আবু
Topic 2: শেখ হাসিনা হাসিনা শেখ পুলিশ উপদেষ্ আওয়ামী লীগ আওয়ামী লীগ নেতা মতামত
Topic 3: বন্যা পরিস্থিতির বন্যা পরিস্থিতির অবনতি পরিস্থিতির অবনতি বন্যা পরিস্থিতির অবনতি যেভাবে পুলিশ গ্রেফত উন্নতি
Topic 4: বাংলাদেশ শিক্ষার্থী না সরক বিসিবি অন্তর্বর্তী পুলিশ পানি জাতিসংঘ বিশ্ববিদ্যালয়
Topic 5: শিক্ষার্থী ছাত্র আন্দোলন বৈষম্যবিরোধী ছাত্র বৈষম্যবিরোধী বিক্ষোভ ছাত্র আন্দোলন পুলিশ বৈষম্যবিরোধী ছাত্র আন্দোলন মিছিল
Topic 6: সংস্কার শিক্ষার্থী কোটা সংস্কার দাবি কোটা সংস্কার দাবি কোটা সংস্কার দাবি সরকারি চাকরি সরকারি চাকরি
Topic 7: আন্দোলন সংস্ক আন্দোলন কোটা সংস্ক আন্দোলন কোটা সংস্ক সংস্ক পুলিশ ছাত্র কোটা শিক্ষার্থী বৈষম্যবিরোধী ছাত্র
Topic 8: ড ইউনূস ড ইউনূস প্রধান প্রধান উপদেষ্ মুহাম্মদ ড মুহাম্মদ না উপদেষ্ মুহাম্মদ ইউনূস
Topic 9: ছাত্রলীগ না পুলিশ কোটা সংস্ক সংস্ক আন্দোলনকারী কোটা সংস্ক আন্দোলনকারী সংস্ক হামলা শিক্ষার্থী অভিযোগ
Topic 10: না সেবা সাঈদ ইন্টারনেট পুলিশ আবু সাঈদ ইন্টারনেট 

In [39]:
# # Tokenize the text
# tokenized_texts = [text.split() for text in texts]

# # Create a dictionary
# dictionary = Dictionary(tokenized_texts)
# corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

# # Define the number of topics
# num_topics = 10  # You can adjust this

# # Train the LDA model
# lda_model = LdaModel(corpus,
#                    num_topics=num_topics,
#                    id2word=dictionary,
#                    random_state=42,
#                    passes=20,
#                    alpha='auto',
#                    per_word_topics=True
#                     )

# # Function to show topics
# def show_topics(lda_model, num_topics=10, top_words=10):
#     for topic_id in range(num_topics):
#         topic_words = lda_model.show_topic(topic_id, topn=top_words)
#         topic_words = " ".join([word for word, prob in topic_words])
#         print(f'Topic {topic_id + 1}: {topic_words}')

# # Show top words for each topic
# print("Top words per topic (using Gensim):")
# show_topics(lda_model, num_topics=num_topics)


# # Function to calculate coherence score
# def calculate_coherence(lda_model, texts, dictionary):
#     coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
#     coherence_score = coherence_model.get_coherence()
#     return coherence_score

# # Calculate coherence score
# coherence_score = calculate_coherence(lda_model, tokenized_texts, dictionary)
# print(f"Coherence score: {coherence_score:.4f}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# # Use TF-IDF to vectorize text
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=None)
# tfidf = tfidf_vectorizer.fit_transform(texts)

# # Define number of topics
# num_topics = 10 # You can adjust this

# # Train the LDA Model
# lda_sklearn = LatentDirichletAllocation(n_components=num_topics, random_state=42)
# lda_sklearn.fit(tfidf)

# # Function to display top words for each topic
# def display_topics(model, feature_names, num_top_words=10):
#   for topic_idx, topic in enumerate(model.components_):
#     print(f"Topic {topic_idx + 1}:", " ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

# # Get feature names
# tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# # Show top words for each topic
# print("\nTop words per topic (using Scikit-learn):")
# display_topics(lda_sklearn, tfidf_feature_names, num_top_words=10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
